# Building a Reasoning AI Agent with ADK & Gemini

## Project Objective

This project showcases the development of a reasoning-capable AI agent built using Google’s Agent Development Kit (ADK) and Gemini large language models. The agent is designed to autonomously interpret queries, make decisions, and invoke external tools to deliver grounded, contextual responses.

Through this implementation, I demonstrate practical expertise in agentic AI design, including environment setup, reasoning loop construction, tool orchestration, and real-time interaction. The notebook serves as an end-to-end demonstration of how to operationalize modern LLM-powered agents for dynamic task execution and intelligent decision-making.

## Overview: Project Highlights

This project demonstrates the end-to-end process of building an intelligent reasoning agent powered by Google’s Agent Development Kit (ADK) and Gemini models. The agent autonomously reasons through queries, invokes external tools, and adapts its responses based on real-time information.

**Project Workflow**

**Environment Setup** — Configured secure API authentication and integrated core dependencies for Gemini-powered agent execution.

**Agent Configuration** — Implemented a reasoning agent with dynamic tool selection (e.g., Google Search) and robust retry logic for rate-limited APIs.

**Live Execution** — Deployed the agent on real-world prompts to showcase reasoning, tool invocation, and decision-making in action.

**Key Concepts** — Illustrated how agents determine when and how to use tools to enhance accuracy and context relevance.

**Why It Matters**
This project highlights practical expertise in agentic AI development—demonstrating the transition from static LLMs to autonomous, action-driven systems capable of reasoning, planning, and executing tasks. The resulting agent exemplifies how modern AI architectures can blend reasoning with real-world utility

## **1. Environment Setup**

### **1.1 Import Libraries**

**Overview:**
This section initializes the core dependencies required to develop a reasoning-capable AI agent using Google’s **Agent Development Kit (ADK)** and **Gemini** models.

**Implementation Summary:**
The imported libraries enable:

* **Agent Development Kit (ADK)** — Provides the foundation for defining autonomous, reasoning-driven agents.
* **Gemini LLM** — Powers the agent’s cognitive and generative capabilities.
* **Google Search Tool** — Integrates real-time information retrieval into the reasoning workflow.
* **Type Utilities & Retry Handlers** — Support robust API communication and structured error management.

**Technical Insight:**
By leveraging ADK’s high-level abstractions, this setup streamlines agent orchestration—enabling a focus on **capability design** and **behavioral logic**, rather than low-level implementation details.

In [4]:
import os
from kaggle_secrets import UserSecretsClient

from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types


from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### **1.2 API Authentication**

**Overview:**
This section implements secure authentication for accessing the **Gemini API**, ensuring credentials are managed according to production-grade security standards.

**Implementation Summary:**

* **Secure Retrieval:** The **UserSecretsClient()** interface is used to fetch the `GOOGLE_API_KEY` from encrypted Kaggle notebook secrets.
* **Environment Management:** The key is stored in `os.environ`, allowing seamless access by the Gemini client across all runtime components.
* **Error Handling:** A `try-except` block is included to surface configuration or permission errors early, improving system robustness and debuggability.

**Technical Insight:**
By externalizing credentials and avoiding hardcoded tokens, this setup enforces **environmental isolation** and **secure authentication workflows**, minimizing exposure risks. This design also ensures reliable and authorized communication with Google’s AI services, reducing potential issues related to **rate limiting** or **API authentication failures**.


In [2]:
try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(f"🔑 Authentication Error: Check your notebook secret configuration. Details: {e}")

✅ Gemini API key setup complete.


### **1.3 Configure Retry Options**

**Overview:**
This section defines a **resilient retry policy** for all Gemini API interactions, ensuring the agent maintains reliability even under transient network or service disruptions.

**Implementation Summary:**

* **Exponential Backoff Strategy:** Configured with an exponential delay (`exp_base=7`), allowing the system to progressively increase wait times between retries and reduce API pressure.
* **Selective Retry Handling:** Retries are triggered only for recoverable HTTP status codes—`429` (rate limiting) and server-side errors (`500`, `503`, `504`)—while client-side errors are excluded to prevent unnecessary requests.
* **Controlled Attempts:** The retry mechanism caps at `5` attempts, with an initial delay of `1s` that scales exponentially across retries.

**Technical Insight:**
Integrating structured retry logic enhances **system robustness** and **fault tolerance**, key characteristics of production-grade AI agents. By applying selective backoff and recovery strategies, the agent remains operational during temporary API outages or throttling events—ensuring consistent task execution and improved uptime in live environments.

In [5]:
retry_config = types.HttpRetryOptions(
    attempts=5,       # Maximum retry attempts
    exp_base=7,       # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these error codes
)

print("✅ Retry configuration initialized.")

✅ Retry configuration initialized.


## **2. Define the Agent**

**Overview:**
This section configures the core **reasoning agent**, defining its model, behavioral logic, and tool ecosystem. The agent’s setup combines structured reasoning with real-time adaptability, forming the foundation of its autonomous intelligence.

**Implementation Summary:**

* **Model Selection:** Utilizes **Gemini 2.5 Flash Lite**, optimized for fast inference while maintaining high reasoning accuracy—ideal for iterative, tool-driven interactions.
* **Tool Integration:** Connects to **Google Search**, enabling access to live, factual data and grounding the agent’s responses in up-to-date information.
* **Behavioral Configuration:** Defines clear **system instructions** that guide decision-making, tone, and when to invoke external tools.
* **Resilience:** Incorporates retry configurations from Section 1.3 to ensure robust communication and consistent performance under variable network conditions.

**Technical Insight:**
Each parameter serves a targeted purpose:

* `name` — Acts as an identifiable handle for logging, tracking, and debugging agent runs.
* `model=Gemini(...)` — Determines the reasoning engine that drives the agent’s cognitive process.
* `description` — Enables the agent to self-contextualize its role and intended tasks.
* `instruction` — Encodes behavioral constraints and tool invocation logic.
* `tools=[google_search]` — Defines the agent’s accessible external capabilities.

**Key Insight:**
By explicitly instructing the agent on when and how to use **Google Search**, the design achieves **context-aware tool orchestration**—allowing the system to autonomously decide when external data is necessary for reasoning or verification, a hallmark of effective agentic AI design.

In [14]:
# --- 2) Agent instructions (GENERAL, DOMAIN-AGNOSTIC) -------------------------
INSTRUCTION = """
You are a reasoning agent. Decide whether to consult external tools (Google Search) based on the query and your uncertainty.

HARD REQUIREMENTS:
- If the query is time-sensitive, factual, likely to change, or you feel uncertain, you MUST call Google Search at least once BEFORE answering.
- If you have NOT used Google Search for this query and any of the above conditions apply, DO NOT answer. Instead, reply exactly with: SEARCH_REQUIRED

When you use tools:
- Synthesize results in your own words.
- Prefer authoritative sources (gov/edu, official orgs, primary docs, reputable encyclopedias).
- Provide 1–3 sources.

Final output format (strict):
- One concise paragraph with the answer.
- Then a 'Sources:' list with 1–3 bullet points (Title — URL) IF tools were used.
- No chain-of-thought or tool UI snippets.

If the query is ambiguous, ask ONE brief clarifying question before proceeding.
"""


In [6]:
root_agent = Agent(
    name="reasoning_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="An agent that can answer questions by reasoning and searching current facts.",
    instruction=INSTRUCTION,
    tools=[google_search],
)

print("✅ Reasoning Agent defined.")

✅ Reasoning Agent defined.


In [46]:
import re, io, sys, contextlib
from IPython.display import display, HTML

# ---------- Utilities to normalize ADK responses ----------
def _to_event_list(ev):
    """Normalize ADK response into a list of events."""
    if ev is None:
        return []
    if isinstance(ev, (list, tuple)):
        return list(ev)
    # Some ADK builds return an object with .events or .history
    for attr in ("events", "history", "items"):
        if hasattr(ev, attr):
            seq = getattr(ev, attr)
            if isinstance(seq, (list, tuple)):
                return list(seq)
    # Fallback: treat as single event
    return [ev]

def _event_text_single(e) -> str:
    """Extract text from a single Event, robustly."""
    # Try content.parts[].text
    try:
        parts = getattr(getattr(e, "content", None), "parts", None)
        if parts:
            for p in parts:
                if getattr(p, "text", None):
                    t = (p.text or "").strip()
                    if t:
                        return t
    except Exception:
        pass
    # Try direct .text
    try:
        t = getattr(e, "text", None)
        if t:
            return t.strip()
    except Exception:
        pass
    # Try candidates[0].content.parts[].text
    try:
        cands = getattr(e, "candidates", None) or []
        for c in cands:
            parts = getattr(getattr(c, "content", None), "parts", None) or []
            for p in parts:
                if getattr(p, "text", None):
                    t = (p.text or "").strip()
                    if t:
                        return t
    except Exception:
        pass
    return ""

def _event_sources_single(e):
    """Extract sources (title, url) from one Event's grounding metadata."""
    out, seen = [], set()
    try:
        gm = getattr(e, "grounding_metadata", None)
        chunks = getattr(gm, "grounding_chunks", None) or []
        for ch in chunks:
            web = getattr(ch, "web", None)
            uri = getattr(web, "uri", None)
            title = getattr(web, "title", None) or "Source"
            if uri and uri not in seen:
                out.append((title, uri))
                seen.add(uri)
            if len(out) >= 5:
                break
    except Exception:
        pass
    return out

URL_RE = re.compile(r"https?://[^\s)>\]]+", re.IGNORECASE)

def _urls_from_text(text: str):
    urls = URL_RE.findall(text or "")
    out, seen = [], set()
    for u in urls:
        if u not in seen:
            out.append(("Source", u))
            seen.add(u)
        if len(out) >= 5:
            break
    return out

# ---------- High-level extractors that work over a list of events ----------
def model_text(ev) -> str:
    """Return the last non-empty model text across the event list."""
    events = _to_event_list(ev)
    last_text = ""
    for e in events:
        t = _event_text_single(e)
        if t:
            last_text = t
    return last_text

def extract_sources(ev):
    """Collect sources across all events; fallback to URLs found in final text."""
    events = _to_event_list(ev)
    srcs, seen = [], set()
    for e in events:
        for t,u in _event_sources_single(e):
            if u not in seen:
                srcs.append((t,u)); seen.add(u)
    if not srcs:
        # fallback: parse URLs from final text
        txt = model_text(ev)
        srcs = _urls_from_text(txt)
    return srcs

def used_web(ev) -> bool:
    """Detect if any event includes grounding chunks (proxy for Search)."""
    events = _to_event_list(ev)
    for e in events:
        try:
            gm = getattr(e, "grounding_metadata", None)
            chunks = getattr(gm, "grounding_chunks", None)
            if chunks:
                return True
        except Exception:
            continue
    return False

# ---------- Output tidying ----------
NAME_RE = re.compile(r"^\*+\s*\*\*(.+?)\*\*", re.IGNORECASE)

def dedupe_bullets(text: str) -> str:
    lines = text.splitlines()
    seen_names = set()
    out_lines = []
    for ln in lines:
        m = NAME_RE.match(ln.strip())
        if m:
            name = m.group(1).strip().lower()
            if name in seen_names:
                continue
            seen_names.add(name)
        out_lines.append(ln)
    return "\n".join(out_lines)

def tidy_paragraphs(text: str) -> str:
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

PREF_AUTH = (
    ".gov", ".edu", "aaai.org", "neurips.cc", "icml.cc", "iclr.cc",
    "cvpr.thecvf.com", "eccv2025.eu", "databricks.com", "worldsummit.ai",
    "aisummit.com", "aiconference.com", "nvidia.com/gtc",
    "acm.org", "ieee.org", "nature.com", "arxiv.org", "wikipedia.org"
)

def is_authoritative(url: str) -> bool:
    u = (url or "").lower()
    return any(dom in u for dom in PREF_AUTH)

def pick_authoritative(sources, k=3):
    auth = [s for s in sources if is_authoritative(s[1])]
    non = [s for s in sources if not is_authoritative(s[1])]
    seen, out = set(), []
    for group in (auth, non):
        for t,u in group:
            if u not in seen:
                out.append((t,u)); seen.add(u)
            if len(out) >= k:
                return out
    return out

def format_final(ev) -> str:
    main = model_text(ev)
    if main:
        main = dedupe_bullets(main)
        main = tidy_paragraphs(main)
    else:
        main = "(No content returned)"

    srcs = extract_sources(ev)
    srcs = pick_authoritative(srcs, k=3)

    if srcs:
        main += "\n\nSources:\n" + "\n".join([f"- {t} — {u}" for t,u in srcs])
    return main

def render_html(text: str):
    html_lines, in_ul = [], False
    for line in text.splitlines():
        if line.startswith("- "):
            if not in_ul:
                html_lines.append("<ul>"); in_ul = True
            html_lines.append(f"<li>{line[2:]}</li>")
        else:
            if in_ul:
                html_lines.append("</ul>"); in_ul = False
            if line.strip():
                html_lines.append(f"<p>{line}</p>")
            else:
                html_lines.append("<br/>")
    if in_ul:
        html_lines.append("</ul>")
    html = "\n".join(html_lines)
    display(HTML(f'<div style="line-height:1.55;font-family:system-ui,Segoe UI,Roboto,Arial,sans-serif">{html}</div>'))

# ---------- Controller: one polished block; no debug spam ----------
async def run_and_print(query: str):
    """
    Uses run_debug (which accepts a prompt) but suppresses its spammy prints.
    If the query is likely retrieval-worthy and we didn't get sources, nudge twice.
    """
    def looks_time_sensitive(q: str) -> bool:
        ql = q.lower()
        return (
            any(w in ql for w in ("last","latest","current","today","schedule","dates","2024","2025","2026","top","best","ranking","conference","summit","workshop"))
        )

    def nudge(q: str, note: str):
        hint = " Prefer official event sites (neurips.cc, icml.cc, aiconference.com, aisummit.com, worldsummit.ai, databricks.com) or reputable listings."
        return (f"{note} Use Google Search to verify and collect 2–3 authoritative sources.{hint} "
                f"Then answer concisely and list the sources.\nQuery: {q}")

    buf = io.StringIO()
    with contextlib.redirect_stdout(buf):
        ev = await runner.run_debug(query)

    # Enforce retrieval + sources only if it looks like it should have them
    text = model_text(ev)
    has_sources = bool(extract_sources(ev))
    tries = 0
    while looks_time_sensitive(query) and not has_sources and tries < 2:
        with contextlib.redirect_stdout(buf):
            ev = await runner.run_debug(nudge(query, "Before answering,"))
        text = model_text(ev)
        has_sources = bool(extract_sources(ev))
        tries += 1

    final_block = format_final(ev)
    render_html(final_block)


In [49]:
import html

def format_final(ev) -> str:
    """Produce a clean, deduped block with a single Sources section."""
    main = model_text(ev)
    if not main:
        return "(No content returned)"
    # Remove any in-text markdown 'Sources' section from the model output
    main = re.sub(r"(?is)(\*\*?Sources:?(\*\*)?.*?$)", "", main).strip()
    main = dedupe_bullets(main)
    main = tidy_paragraphs(main)

    srcs = extract_sources(ev)
    srcs = pick_authoritative(srcs, k=3)

    if srcs:
        main += "\n\nSources:\n" + "\n".join([f"- {t} — {u}" for t,u in srcs])
    return main.strip()


def render_html(text: str):
    """Convert basic markdown to HTML and display nicely."""
    # Escape any stray HTML first
    text = html.escape(text)

    # Bold **text**
    text = re.sub(r"\*\*(.*?)\*\*", r"<b>\1</b>", text)

    # Convert * bullets to <ul><li>
    lines = text.splitlines()
    html_lines, in_ul = [], False
    for line in lines:
        if line.strip().startswith("* "):
            if not in_ul:
                html_lines.append("<ul>"); in_ul = True
            html_lines.append(f"<li>{line.strip()[2:]}</li>")
        else:
            if in_ul:
                html_lines.append("</ul>"); in_ul = False
            if line.strip():
                html_lines.append(f"<p>{line}</p>")
            else:
                html_lines.append("<br/>")
    if in_ul:
        html_lines.append("</ul>")
    html_block = "\n".join(html_lines)

    display(
        HTML(
            f'<div style="line-height:1.6; font-family:system-ui,Segoe UI,Roboto,Arial,sans-serif;">'
            f"{html_block}</div>"
        )
    )


## **3. Execute the Agent**

**Overview:**
This phase validates the agent’s end-to-end behavior on real-world prompts—capturing how it reasons, decides to invoke tools, and composes grounded answers.

**Execution Focus:**

* **Query Processing:** Evaluate how inputs propagate through the reasoning loop and state updates.
* **Tool Orchestration:** Inspect when and why **Google Search** is invoked to retrieve or verify up-to-date context.
* **Answer Synthesis:** Confirm that retrieved evidence is integrated into concise, coherent outputs.
* **Full Traceability:** Record the complete decision path for post-run analysis and debugging.

**Diagnostics & Observability:**
`runner.run_debug()` captures a full execution trace, including:

* Internal reasoning checkpoints and intermediate thoughts
* Tool invocation metadata (parameters, responses, latencies)
* Final response construction steps
* Structured logs suitable for performance tuning and error analysis

**Outcome:**
These diagnostics establish an auditable reasoning trail and demonstrate production-minded **observability**—making it straightforward to tune prompts, adjust tool policies, and refine retry/backoff parameters under real traffic patterns.

In [18]:
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner instantiated.")

✅ Runner instantiated.


## **3.1 Current-Affairs Query (Example 1)**

**Prompt:** *“Who won the last soccer World Cup?”*

**Intent:**
Demonstrate the agent’s ability to handle **time-sensitive, factual queries** that require **real-time retrieval** rather than relying on static training data.

**Expected Behavior:**

* **Temporal Awareness:** Detects that the answer changes over time and that retrieval is required.
* **Tool Use:** Invokes **Google Search** to fetch authoritative, recent sources.
* **Grounding & Synthesis:** Consolidates retrieved evidence into a clear, unambiguous answer.
* **Citations/Traceability:** Surfaces the reasoning trace and tool responses in debug logs for verification.

**Evaluation Signals (what this run proves):**

* The **decision policy** correctly routes to the Search tool for dynamic facts.
* The **reasoning loop** integrates external evidence before generating the final reply.
* The **retry/backoff** policy holds under real API conditions without silent failure.
* The **observability** layer (debug trace) provides end-to-end provenance.

**Example Invocation (conceptual):**

* **Input:** “Who won the last soccer World Cup?”
* **Agent Actions:**

  1. Classifies as time-sensitive → 2) Calls `google_search` with focused query →
  2. Parses recent results → 4) Generates a grounded answer with source-backed confidence.

**Acceptance Criteria:**

* Final answer names the **correct champion** and **edition/year**.
* Debug trace shows at least **one successful tool call** and **no client-error retries**.
* Latency remains within acceptable bounds for interactive use (validated in logs).

**Why this example belongs in the showcase:**
It isolates a core competency of **agentic systems**—**context-aware tool orchestration**—and demonstrates that the agent reliably transitions from reasoning to action to verified output on a changing factual landscape.

In [51]:
await run_and_print("Who won the last soccer World Cup?")


## **3.2 Research-Oriented Query (Example 2)**

**Prompt:** *“What are the top AI conferences in 2025?”*

**Intent:**
Demonstrate the agent’s ability to handle **open-ended research queries** that demand **multi-source retrieval**, **reasoning over trends**, and **synthesized narrative formation**—rather than merely returning discrete facts.

**Expected Behavior:**

* **Query Characterization:** Detects that the user is requesting a structured overview of events in 2025, not just a single fact.
* **Comprehensive Retrieval:** Executes one or more calls to the search tool to harvest up-to-date listings, major announcements, and schedule shifts for AI conferences.
* **Reasoning & Synthesis:**

  * Identifies major conferences (e.g., NeurIPS 2025, ICML 2025, CVPR 2025) and any new emerging formats.
  * Highlights key trends (e.g., increased focus on generative AI ethics, agent-based models, hybrid on-site/virtual formats).
  * Organizes the output into clear sections (e.g., Conference Overview, Key Themes, Emerging Formats, Why It Matters).
* **Traceability:** The debug log shows multiple search tool invocations, their results, and reasoning steps combining them into the final answer.

**Evaluation Signals (what this run demonstrates):**

* The agent transitions from retrieval to **analysis** (not just listing).
* It shows **pattern identification** (e.g., “In 2025 the theme across top conferences is…”) rather than static reporting.
* It maintains **robustness** under potentially variable search results (e.g., new or shifted conferences).
* The observability layer provides a full trace of tool calls, intermediate logic, and final synthesis.

**Acceptance Criteria:**

* Final answer enumerates 4-6 key AI conferences in 2025 with brief descriptions (location/form, highlights, relevance).
* The reasoning includes topical themes and signals why each conference is notable in 2025.
* No uncited leaps; evidence in trace supports conclusions.
* Execution logs show retrieval depth (i.e., more than one source) and coherent reasoning.

**Why this example belongs in the showcase:**
It underscores the agent’s **higher-order capabilities**: not just retrieving facts, but **connecting data**, **identifying trends**, and **communicating structured insights**—a clear demonstration of agentic AI in a practical, modern domain.


In [50]:
await run_and_print("What are the top AI conferences in 2025?")


## **4. Agent Reasoning and Action Flow**

**Overview:**
The agent’s decision process follows a **reasoning-first architecture**. Upon receiving a query, it evaluates whether sufficient context exists within its internal knowledge. If uncertainty is detected, it autonomously invokes the **Google Search tool** to retrieve current and relevant data.

**Workflow Summary:**

1. **Intent Recognition:** Classifies the query as factual, time-sensitive, or analytical.
2. **Reasoning Pass:** Determines whether its internal context can yield a reliable answer.
3. **Tool Invocation:** When external data is required, the agent issues structured search queries.
4. **Evidence Integration:** Synthesizes retrieved results into a coherent, well-grounded response.
5. **Traceable Output:** Each reasoning step, tool call, and synthesis stage is captured for observability and debugging.

**Key Insight:**
This **traceable reasoning loop** exemplifies modern agentic AI—systems that not only generate answers but also **decide, act, and explain** their decision path. The integration of reasoning transparency and tool autonomy reflects production-grade design principles for intelligent, accountable systems.

## **5. Project Summary**

**Outcome Overview:**
This project delivers a complete demonstration of building and deploying a **reasoning-capable AI agent** using **Google’s Agent Development Kit (ADK)** and **Gemini** models.

**Key Achievements:**

* **End-to-End Implementation:** Covered environment setup, secure API authentication, agent definition, retry resilience, and live query execution.
* **Demonstrated Reasoning Capabilities:** The agent autonomously determined when to use tools, executed real-time searches, and synthesized results.
* **Transparent Workflows:** Every decision and tool call was fully traceable through debug outputs.
* **Reproducible Design:** All configurations and outputs are documented for consistent replication or extension.

**Professional Impact:**
This notebook exemplifies hands-on **agentic AI proficiency**, bridging model reasoning, tool orchestration, and observability. It serves as a strong portfolio artifact showcasing the ability to **design, implement, and interpret intelligent autonomous systems**.

## 🔗 Next Steps: From Reasoning to Multi-Agent Architectures

This notebook focused on designing a single reasoning agent using Google’s ADK and Gemini — demonstrating tool use, action selection, and grounded reasoning.

In the next phase, I extend this foundation to explore **multi-agent architectures**, where multiple specialized agents (Planner, Researcher, Summarizer) collaborate to solve complex tasks.

👉 [Continue to the next notebook: *Multi-Agent Architectures with ADK & Gemini*](https://www.kaggle.com/code/kanikaw/agentic-ai-architectures-sequential-parallel-l)
